#### Libraries

In [1]:
from collections import Counter
import random
import os

#### Parametros

In [2]:
# Definição da tabela de páginas e da memória física
PAGE_SIZE = 4096 # Tamanho da página em bytes
BACKING_STORE_SIZE = 256 # Tamanho do backing store em bytes
page_table = [None] * 256
queue = []
physical_memory = {}
requested_pages = []
frame = 256
fault = 0
top = 0

#### Funções

In [3]:
# Gerar lista de enderecos

def generateAddress(address_quantity=1000):
    address_list = []
    for i in range(address_quantity):
        address_bits = random.getrandbits(16)
        address_list.append(address_bits)          
    return address_list

In [4]:
# Salvar lista de enderecos em um arquivo

def saveAddress(adress_list):
    with open("addresses.txt", "w") as f:
        for address in adress_list:
            f.write("%s\n" % address)

In [5]:
def load_logical_addresses(filename):
    with open(filename) as file:
        return [int(line.strip()) for line in file]

In [6]:
def read_from_backing_store(page_num):
    # Lê a página correspondente do backing store
    with open(backing_store_file, "rb") as f:
        f.seek(page_num * PAGE_SIZE)
        page_data = f.read(PAGE_SIZE)
    return page_data


In [7]:
def pageFaultHandler(page_number, physical_memory):
    frame_number = -1
    if int(page_number) < 256:
        for i in range(256):
            if i in physical_memory.keys():
                continue
            else:
                frame_number = str(i)
                break

        physical_memory[int(frame_number)] = []

        data = str(read_from_backing_store(page_number))
        print('Page \"' + str(page_number) + '\" has this data in the backing store: ')
        print(physical_memory[int(frame_number)])
    else:
        print('Page \"' + page_number + '\" is out of bound!')
        return frame_number

In [8]:
def translate_address(logical_address):
    page_number = ((logical_address & 65535) >> 8) #8 bits to the right
    page_offset = logical_address & 255 #last 8 bits
    frame_number = page_table2[page_number]
    if frame_number is None:
        print(f"Page fault para página {page_number}")
        frame_number = pageFaultHandler(page_number, physical_memory)
        page_table[page_number] = frame_number
    physical_address = "{0:08b}".format(int(page_number)) + "{0:08b}".format(page_offset)
    physical_address = int(physical_address, 2)
    
    physical_memory[frame_number] = page_offset
    return physical_address, page_number

#### Gerando txt com endereços 1000 endereços de 16 bits

In [9]:
address_file = generateAddress()
saveAddress(address_file)

##### Cria page table

In [10]:
for i in (range(len(page_table))):
    page_table[i] = random.getrandbits(8)

histogram = Counter(page_table)
page_table2 = [n if histogram[n] <= 1 else None for n in page_table]

#### Cria arquivo de backing store

In [11]:
# Cria o arquivo de backing store
backing_store_file = "backing_store.bin"
if not os.path.exists(backing_store_file):
    with open(backing_store_file, "wb") as f:
        f.seek(BACKING_STORE_SIZE - 1)
        f.write(b"\0")


#### Traducao de enderecos

In [12]:
logical_addresses = load_logical_addresses("addresses.txt")
for logical_address in logical_addresses:
    physical_address, page_number = translate_address(logical_address)
    requested_pages.append(page_number)
    print(f"Endereço lógico: {logical_address}, Endereço físico: {physical_address}")

Page fault para página 92
Page "92" has this data in the backing store: 
[]
Endereço lógico: 23698, Endereço físico: 23698
Page fault para página 212
Page "212" has this data in the backing store: 
[]
Endereço lógico: 54320, Endereço físico: 54320
Page fault para página 161
Page "161" has this data in the backing store: 
[]
Endereço lógico: 41468, Endereço físico: 41468
Page fault para página 196
Page "196" has this data in the backing store: 
[]
Endereço lógico: 50376, Endereço físico: 50376
Page fault para página 85
Page "85" has this data in the backing store: 
[]
Endereço lógico: 21794, Endereço físico: 21794
Page fault para página 133
Page "133" has this data in the backing store: 
[]
Endereço lógico: 34176, Endereço físico: 34176
Endereço lógico: 42387, Endereço físico: 42387
Page fault para página 118
Page "118" has this data in the backing store: 
[]
Endereço lógico: 30336, Endereço físico: 30336
Page fault para página 81
Page "81" has this data in the backing store: 
[]
Endere

#### FIFO

In [13]:
print("\nRequested Pages|Page Table ",end='')
for i in range(frame):
    print(i, end=' ')
print("\n   \n")
for i in requested_pages:
    if i not in queue:
        if (len(queue) < frame):
            queue.append(i)
        else:
            queue[top] = i
            top = (top+1) % frame
        fault += 1
        page_fault = 'Page fault happened!'
    else:
        page_fault = 'Page fault didnt happen!'
    print("   %d\t\t" % i, end='')
    for x in queue:
        print(x, end=' ')
    for x in range(3-len(queue)):
        print(' ', end=' ')
    print(" %s" % page_fault)
print('\n% Page Faults: {}'.format(fault/len(requested_pages)))


Requested Pages|Page Table 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 
   

   92		92      Page fault happened!
   212		92 212  